In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
%cd engg680_2024_fall

/content/drive/MyDrive/engg680_2024_fall


In [ ]:
%cd Project

/content/drive/MyDrive/engg680_2024_fall/Project


# **ENGG680 - Introduction to Digital Engineering**
## ***PROJECT: Automated Detection of Roads and Pathways in Aerial Imagery Using Deep Learning (U-Net Model)***





## Preliminary: Certificate of Work


*We, the undersigned, certify that this is our own work, which has been done expressly for this course, either without the assistance of any other party or where appropriate we have acknowledged the work of others. Further, we have read and understood the section in the university calendar on plagiarism/cheating/other academic misconduct and we are aware of the implications thereof. We request that the total mark for this assignment be distributed as follows among group members:*

|          | First Name | Last Name | Signature (Full Name, Date) | Hours | Contribution % |
|----------|------------|-----------|-----------------------------|-------|----------------|
| Member 1: | Akinsegun | Ademiluwa | Akinsegun Ademiluwa, 2024-12-09 | 72 Hours | 12.5 |
| Member 2: | Ridwan | Sharafadeen | Ridwan Sharafadeen, 2024-12-09 | 72 Hours | 12.5 |
| Member 3: | Cyprain | Okafor | Cyprain Okafor, 2024-12-09 |  72 Hours | 12.5 |
| Member 4: | Onyeka | Emecheta | Onyeka Emecheta, 2024-12-09 | 72 Hours | 12.5 |
| Member 5: | Akolawole | Bode-Fakunle | Akolawole Bode-Fakunle, 2024-12-09 | 72 Hours | 12.5 |
| Member 6: | Victor | Fajonyomi | Victor Fajonyomi, 2024-12-09 | 72 Hours | 12.5 |
| Member 7: | Genevieve | Aluziwe | Genevieve Aluziwe, 2024-12-09 |  72 Hours | 12.5 |
| Member 8: | David | Olubiyi | David Olubiyi, 2024-12-09 | 72 Hours | 12.5 |

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from albumentations import HorizontalFlip, VerticalFlip, RandomRotate90, Compose

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
# Define paths based on your project structure:
aerial_folder = "C:/Users/AL-WASI/Desktop/ML/FUOtuoke/University of Calgary/Aerial"
image_folder = os.path.join(aerial_folder, "road_path")
img_path = os.path.join(image_folder, "img.png")
mask_path = os.path.join(image_folder, "label.png")


In [ ]:
# Load mask function with binary normalization:
def load_mask(mask_path):
    if not os.path.exists(mask_path):
        print(f"Error: The file at {mask_path} does not exist.")
        return None
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print("Error: Failed to load the mask. Check the file path or file integrity.")
        return None
    mask = cv2.resize(mask, (256, 256))


In [ ]:
    # Binarize mask: set all values greater than 0 to 1 (assuming binary segmentation):
    mask[mask > 0] = 1

In [ ]:
    # Convert mask to one-hot encoded format
    mask_one_hot = to_categorical(mask, num_classes=2)
    print("Mask one-hot shape:", mask_one_hot.shape)  # Should be (256, 256, 2)
    return mask_one_hot

In [ ]:
# Set paths:
aerial_folder = "C:/Users/AL-WASI/Desktop/ML/FUOtuoke/University of Calgary/Aerial"
image_folder = os.path.join(aerial_folder, "road_path")
img_path = os.path.join(image_folder, "img.png")
mask_path = os.path.join(image_folder, "label.png")

In [ ]:
# Load images and masks:
original_image = load_image(img_path)
original_mask = load_mask(mask_path)

if original_image is not None and original_mask is not None:
    print("Image and mask loaded and processed successfully.")
else:
    print("Failed to load image or mask.")

In [ ]:
# Augmentation:
def augment_data(image, mask, num_augmented=10):
    aug = Compose([
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        RandomRotate90(p=0.5)
    ])
    images, masks = [image], [mask]
    for _ in range(num_augmented):
        augmented = aug(image=image, mask=mask)
        images.append(augmented['image'])
        masks.append(augmented['mask'])
    return np.array(images), np.array(masks)

augmented_images, augmented_masks = augment_data(original_image, original_mask)

In [ ]:
# Define a subset of augmented images and masks for testing
# Assuming you have enough samples, you can set aside some for testing:
split_index = int(0.8 * len(augmented_images))  # 80% for training, 20% for testing
train_images, test_images = augmented_images[:split_index], augmented_images[split_index:]
train_masks, test_masks = augmented_masks[:split_index], augmented_masks[split_index:]

In [ ]:

# Define UNet Model
def unet_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    # Down-sampling layers
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    # Up-sampling layers
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3], axis=-1)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

    outputs = Conv2D(2, (1, 1), activation='softmax')(conv9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# Initialize and train the model
model = unet_model()

# Training the model using the augmented data
#history = model.fit(augmented_images, augmented_masks, batch_size=4, epochs=10, validation_split=0.2)
# Re-train model with training split (optional if already trained)
history = model.fit(train_images, train_masks, batch_size=4, epochs=10, validation_split=0.2)
